### This script will load up entity profiles. Note, it will ONLY work on raiders8.

In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import ujson as json
import jsonlines
import os,sys
sys.path.append('/dfs/scratch1/simran/tutorial/bootleg-internal/notebooks')
sys.path.append('/dfs/scratch1/simran/tutorial/bootleg-internal/')
import time
import multiprocessing
from tabulate import tabulate
from collections import defaultdict
import os,sys
import pandas as pd
import jsonlines
from tqdm import tqdm
from collections import defaultdict
import nltk
nltk.download('stopwords')
import utils_not_for_public as du


/dfs/scratch1/simran/pkg/.pkg_venv/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/dfs/scratch1/simran/pkg/.pkg_venv/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     /lfs/1/simran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def load_rel_names():
    return json.load(open("/dfs/scratch1/simran/bootleg_downstream/archive/wikidata_mappings/pid_names.json"))

In [5]:
# Loads entity profiles
# es -> entity_symbols stores QID -> title and QID->candidates
# esp -> entity_symbols_plus stores QID->structural resource and QID->candidates; esp is more memory efficient and should be used in distributed processes
# type_vocab -> hyena (coarse) type vocabulary
# typeid2typename -> hyena type id to type name
# type_vocab_wd -> wikidata (fine) type vocabulary
# typeid2typename_wd -> wikidata type id to type QID name
# alias_qid_count is alias->qid->number of times appears in training
# typename2title: # As wikidata types go from type id -> QID name; this goes from QID name -> QID title for the type

####### FILTERED ########
#es, esp, type_vocab, typeid2typename, type_vocab_wd, typeid2typename_wd, alias_qid_count = du.load_esp("wiki", overwrite=True) 
es, esp, type_vocab, typeid2typename, type_vocab_wd, typeid2typename_wd, alias_qid_count = du.load_esp("wiki", 
    kg_adj='kg_adj_complete.txt',
    kg_triples='kg_triples.txt',
    hy_vocab='hyena_vocab.json',
    hy_types='hyena_types_wiki_all_aug.json',
    wd_vocab='wikidata_to_typeid.json',
    wd_types='wikidata_types_wiki_filt.json', overwrite=False)   

Loading entity_symbols from /dfs/scratch0/lorr1/data_prep/data/wiki_0906_pg/_cache_wiki_0906_pg/wiki0516/entity_db/entity_mappings
/dfs/scratch0/lorr1/data_prep/data/wiki_0906_pg/_cache_wiki_0906_pg/wiki0516/entity_db/entity_mappings


FileNotFoundError: [Errno 2] No such file or directory: '/dfs/scratch0/lorr1/data_prep/data/wiki_0906_pg/_cache_wiki_0906_pg/wiki0516/entity_db/entity_mappings/config.json'

In [5]:
# es, esp, type_vocab, typeid2typename, type_vocab_wd, typeid2typename_wd, alias_qid_count = du.load_esp("wiki", 
#     kg_adj='kg_adj_0905.txt',
#     kg_triples='kg_triples_0905.txt',
#     hy_vocab='hyena_vocab.json',
#     hy_types='hyena_types_0905.json',
#     wd_vocab='wikidata_to_typeid_0905.json',
#     wd_types='wikidata_types_0905.json', overwrite=False)

In [6]:
# print(typeid2typename_wd)

In [20]:
typename2title_wd = du.load_wikidata_typename_map()
rel_to_name = load_rel_names()

In [ ]:
for k, v in rel_to_name.items():
    if 'country' in v:
        print(k,v)

In [22]:
typeqid_to_id = json.load(open("/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_cidr_model/bootleg_downloads/emb_data/wikidata_types_wiki_filt.json"))


In [23]:
typeqid_to_id['Q8104']

[2543]

In [ ]:
qid = "Q265"
qid2 = "Q5145844"
alias = "uzbekistan"

# To access the title of a QID
title = es.get_title(qid)
print(f"Title {title}")
print()
title = es.get_title(qid2)
print(f"Title {title}")
print()

# Get wikidata types for a qid 
types = du.get_qid_types(qid, esp, typeid2typename_wd, wikidata=True)
print(f"Types {types}")
print()

# To get wikidata types in a nice format
types = [typename2title_wd[t] for t in types]
print(f"Types Pretty {types}")
print()

# Get heyan types for a qid 
types = du.get_qid_types(qid, esp, typeid2typename, wikidata=False)
print(f"Hyena Types {types}")

# Get one of the relationships between two qids
rel = esp.get_relation_name(esp.get_relation(qid, qid2))
rel = rel_to_name.get(rel, rel)
print(f"Rel {rel}")

# If there are multiple relationships between two qids (both directions)
rels = [esp.get_relation_name(r) for r in esp.get_all_relations(qid, qid2)]
rels = [rel_to_name.get(rel, rel) for rel in rels]
print(f"Rels {rels}")

# Get all connected qids for a give qid
related_qids = du.get_related_qids(qid, esp)
print(f"Related QIDs {related_qids}")
print([es.get_title(r) for r in related_qids])

In [12]:
# # pushed to github 

# qids = ['Q33999', 'Q3501317', 'Q177220', 'Q4610556', 'Q131524', 'Q8246794', 'Q488205']
# for qid in qids:
#     print(typename2title_wd[qid])

In [13]:
typeid2typename_wd[531]

'Q2661677'

In [26]:
# BY ALIAS: 
alias = "uzbekistan"
try:
    # To get qid candidates of an alias
    cands = esp.get_qid_cands(alias)
    print(f"Cands {cands}")
    print([es.get_title(qid) for qid in cands])
except:
    print("no candidates")
    pass



Cands ['Q265', 'Q487321', 'Q484578', 'Q3247955', 'Q3328961', 'Q1640414', 'Q2118259', 'Q1769173', 'Q3329262', 'Q3328738', 'Q851535', 'Q2116762', 'Q2628681', 'Q7904660', 'Q853837', 'Q586113', 'Q483774', 'Q1837521', 'Q7904680', 'Q7904675', 'Q17747236', 'Q1544234', 'Q39045943', 'Q30716126', 'Q259319']
['Uzbekistan', 'Uzbekistan national football team', 'Uzbek Soviet Socialist Republic', 'Uzbekistan national under-23 football team', 'Uzbekistan national under-20 football team', 'Uzbek cuisine', 'Uzbekistan Davis Cup team', 'Uzbekistan Fed Cup team', 'Uzbekistan national under-17 football team', 'Music of Uzbekistan', "Uzbekistan women's national football team", 'Uzbekistan national futsal team', 'Uzbekistan national basketball team', 'Uzbekistan at the Hopman Cup', 'Uzbekistan at the 2012 Summer Olympics', 'LGBT rights in Uzbekistan', 'Flag of Uzbekistan', 'Uzbekistan at the 2010 Summer Youth Olympics', 'Uzbekistan at the Paralympics', 'Uzbekistan national beach soccer team', 'Uzbekistan at

# ADD ALIASES PLAYGROUND 

In [15]:
# # Load title mapping (helps to debug)
# q2t = json.load(open("/dfs/scratch0/lorr1/contextual-embeddings/aug_data/wiki_prn_0516_lfs/entity_dump/qid2title.json"))

# input_dir = "/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_09042020/bootleg_09042020"  # FILL IN 
# cand_map = f'{input_dir}/wiki_cand_map.json'
# cand_map2 = f'{input_dir}/wiki_cand_map2.json'
# a2q = json.load(open(cand_map))

In [16]:
# # get qid of interest
# word = "us"
# try:
#     # To get qid candidates of an alias
#     cands = esp.get_qid_cands(word)
#     print(f"Cands {cands}")
#     print([es.get_title(qid) for qid in cands])
# except:
#     print("no candidates")
#     pass

# qid_word = esp.get_qid_cands(word)[0]
# print(qid_word)

In [17]:
# # Test if acronym is in the alias set
# # IF THE CORRECT CANDIDATE IS NOT THERE, ADD THE TUPLE WITH [qid, score]
# acronym = "u.s."
# print([q2t[q[0]] for q in a2q.get(acronym, [])])

In [18]:
# # TAKE THE QID AAND A HIGH score VALUE TO MAKE SURE IT'S USED
# if acronym in a2q:
#     a2q[acronym].append([qid_word, 10000])
# else:
#     a2q[acronym] = [[qid_word, 10000]]
# # # REPEAT FOR MORE ACRONYMS
# # ...

In [19]:
# json.save(a2q, open(cand_map2, "w"))
# USE cand_map2 file for mention extraction